# Text Mining Modelling

### Import Library

Disini saya akan melakukan import terhadap beberapa library yang akan dipakai untuk memanggil function yang ada di dalam library tersebut.

Disini saya juga melakukan download 'stopwords', 'punkt', dan 'wordnet' yang memang dibutuhkan untuk proses analisis data lebih lanjut.

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

### Data Exploration

Sebelum melakukan analisa lebih jauh, alangkah baiknya langkah pertama yang harus dilakukan yaitu membaca dataset yang bernama data_1A.csv

In [ ]:
df1 = pd.read_csv('/content/data_1A.csv')

Dapat dilihat 5 data teratas dari dataset yang dimiliki

In [ ]:
df1.head(5)

,Unnamed: 0,text,label
0,0,The Theory of Everything Review Stephen Hawkin...,Books
1,1,Computer Networks: A Top - Down Approach About...,Books
2,2,Sajani Premium Quality Brown Wooden Coat Hange...,Household
3,3,Bosch Lifestyle MCM3501M 800-Watt Food Process...,Household
4,4,Secret Wish Women's Navy-Blue Towel Bathrobe (...,Household


Dapat dilihat 5 data terakhir dari dataset yang dimiliki

In [ ]:
df1.tail(5)

,Unnamed: 0,text,label
12601,12601,Lotus Makeup Ecostay Insta Smooth Perfecting P...,Household
12602,12602,The Subtle Art of Not Giving a F*ck Review “Re...,Books
12603,12603,Elevanto Premium Collection 3/4Th Sleeve Terry...,Household
12604,12604,WD My Passport 4TB Portable External Hard Driv...,Electronics
12605,12605,Storite 16 Foot 3.5 mm Male To 3.5 mm Female J...,Electronics


Dengan menjalankan kode di bawah, anda dapat melihat berbagai informasi seputar dataset, mulai dari jumlah kolom beserta dengan namanya, jumlah data serta tipe data dari setiap variabel.

Kalau dilihat secara teliti di variable 'text' jumlah data yang ada itu hanya 12605 sementara jumlah data itu sebesar 12606, hal ini berarti terdapat 1 missing value di variabel 'text'.

Biasanya jika missing value hanya ada sedikit ini tidak akan terlalu mempengaruhi data, namun disini saya akan memperbaiki agar tidak ada missing value sama sekali.

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12606 entries, 0 to 12605
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  12606 non-null  int64 
 1   text        12605 non-null  object
 2   label       12606 non-null  object
dtypes: int64(1), object(2)
memory usage: 295.6+ KB


Setelah saya melihat ketiga variabel yang ada di dataset, Saya beranggapan bahwa variabel bernama 'Unnamed: 0' ini keberadaannya di dataset ada atau tidak itu gk terlalu mempengaruhi dataset.

Hal ini dikarenakan variabel 'Unnamed: 0' hanya berisi angka yang menunjukkan banyaknya data.

In [ ]:
df1.drop(columns=['Unnamed: 0'], inplace=True)
df1.head(5)

,text,label
0,The Theory of Everything Review Stephen Hawkin...,Books
1,Computer Networks: A Top - Down Approach About...,Books
2,Sajani Premium Quality Brown Wooden Coat Hange...,Household
3,Bosch Lifestyle MCM3501M 800-Watt Food Process...,Household
4,Secret Wish Women's Navy-Blue Towel Bathrobe (...,Household


### Data Preparation / Data Cleaning

Setelah Data Exploration selesai, maka tahapan selanjutnya itu masuk ke Data Preparation dan Data Cleaning.

Proses ini dilakukan agar data yang dimiliki itu menjadi bersih dan lebih siap untuk digunakan pada proses analisis data atau membuat model Machine Learning.

Karena sebelumnya saya menemukan bahwa terdapat 1 missing value di variabel 'text' oleh karena itu saya memperbaikinya disini. Dapat dilihat output yang dihasilkan bahwa sudah sama sekali tidak terdapat missing value di kedua variabel.

In [ ]:
df1 = df1.fillna('')
df1.isnull().sum()

text     0
label    0
dtype: int64

Di bawah ini merupakan function Cleaning data yang saya buat sendiri, di dalamnya terdapat berbagai macam proses data cleaning. Seperti yang dapat dilihat pada code di bawah ini

In [ ]:
def cleaning(df):
    # Ubah ke Lowercase
    clean_text = df.str.casefold()
    # Menghapus Angka
    clean_text = [re.sub(r'\d+', '', i ) for i in clean_text]
    # Menghapus tanda baca
    clean_text = [re.sub(r'[^\w]', ' ', i) for i in clean_text]
    # Menghapus Spasi Double
    clean_text = [re.sub(r'\s+', ' ', i) for i in clean_text]
    # Menghapus stopwords
    stop_words = set(stopwords.words('english'))
    clean_text = [' '.join([word for word in word_tokenize(text) if word.lower() not in stop_words]) for text in clean_text]
    # Tokenisasi
    clean_text = [word_tokenize(text) for text in clean_text]
    # Lemmatisasi
    lemmatizer = WordNetLemmatizer()
    clean_text = [[lemmatizer.lemmatize(word) for word in text] for text in clean_text]
    # Menggabungkan kata per kata menjadi satu kalimat
    clean_text = [' '.join(text) for text in clean_text]

    return clean_text

Setelah function cleaning selesai dibuat, maka tahapan selanjutnya itu mendefinisikan varabel di dataset mana yang ingin dibersihkan.

Dalam kasus saya yang ingin dibersihkan itu variabel bernama 'text', nah hasil data cleansing dari 'text' ini akan saya masukkan ke sebuah variabel baru bernama 'clean_text'

In [ ]:
df1['clean_text'] = cleaning(df1['text'].apply(str))
df1.head()

,text,label,clean_text
0,The Theory of Everything Review Stephen Hawkin...,Books,theory everything review stephen hawking theor...
1,Computer Networks: A Top - Down Approach About...,Books,computer network top approach author behrouz f...
2,Sajani Premium Quality Brown Wooden Coat Hange...,Household,sajani premium quality brown wooden coat hange...
3,Bosch Lifestyle MCM3501M 800-Watt Food Process...,Household,bosch lifestyle mcmm watt food processor black...
4,Secret Wish Women's Navy-Blue Towel Bathrobe (...,Household,secret wish woman navy blue towel bathrobe fre...


Dikarenakan proses selanjutnya itu merupakan proses Vectorization dan membuat model Machine Learning sederhana oleh karena itu Saya memisahkan mana yang menjadi variabel X dan mana yang menjadi variabel Y, setelah ditentukan maka proses selanjutnya dilakukan train_test_split terhadap data dengan test data sebesar 20% sedangkan training data sebesar 80%.

In [ ]:
X = df1['clean_text']
y = df1['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Count Vectorizer

CountVectorizer adalah salah satu metode vectorization yang digunakan dalam pemrosesan teks untuk mengubah teks menjadi representasi numerik. Metode ini menghitung frekuensi kemunculan kata-kata dalam teks dan mengonversinya menjadi vektor numerik.

Jadi, CountVectorizer mengambil kumpulan dokumen teks dan menghasilkan matriks di mana setiap barisnya mewakili satu dokumen dan setiap kolom mewakili kata yang ada dalam seluruh dokumen. Nilai di setiap sel matriks menunjukkan berapa kali kata tertentu muncul dalam dokumen yang sesuai.

Di bawah ini dilakukan proses Vectorization dengan menggunakan metode Count Vectorizer dimana:

X_train_count_vector.shape akan memberikan informasi tentang jumlah baris dan kolom dalam matriks X_train_count_vector, yaitu jumlah dokumen training dan jumlah kata dalam vocabulary, sedangkan

X_test_count_vector.shape akan memberikan informasi serupa untuk matriks X_test_count_vector, yaitu jumlah dokumen testing dan jumlah kata dalam vocabulary yang sama dengan yang ada dalam matriks X_train_count_vector.

In [ ]:
count_vector = CountVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5)
# default lowercase = True, analyser = True, ngram_range=(1,1)

count_vector.fit(X_train)
X_train_count_vector = count_vector.transform(X_train)
X_test_count_vector = count_vector.transform(X_test)

X_train_count_vector.shape, X_test_count_vector.shape

((10084, 28146), (2522, 28146))

Setelah Count Vectorizer sudah didapatkan, maka tahapan selanjutnya yaitu menggunakan Count Vector tersebut ke dalam 2 model Machine Learning disini Saya menggunakan model Logistic Regression dan Multinomial Naive Bayes.

### Logictic Regression With Count Vector

Berikut ini dapat dilihat akurasi dari model Logistic Regression beserta dengan classification report untuk setiap multiclass yang ada di dalam dataset.

In [ ]:
lr_cv = LogisticRegression(max_iter=5000)
lr_cv.fit(X_train_count_vector, y_train)
pred_lr_cv = lr_cv.predict(X_test_count_vector)

print(accuracy_score(y_test, pred_lr_cv))
print(classification_report(y_test, pred_lr_cv))

0.9504361617763679
                        precision    recall  f1-score   support

                 Books       0.93      0.96      0.94       586
Clothing & Accessories       0.96      0.95      0.96       466
           Electronics       0.94      0.92      0.93       506
             Household       0.96      0.96      0.96       964

              accuracy                           0.95      2522
             macro avg       0.95      0.95      0.95      2522
          weighted avg       0.95      0.95      0.95      2522



### Multinomial Naive Bayes With Count Vector

Berikut ini dapat dilihat akurasi dari model Multinomial Naive Bayes beserta dengan classification report untuk setiap multiclass yang ada di dalam dataset.

In [ ]:
mnb = MultinomialNB()

pipeline = Pipeline([('count_vector', CountVectorizer(max_df=100)),
                     ('mnb', MultinomialNB())])

pipeline.fit(X_train, y_train)
pred_NB_cv = pipeline.predict(X_test)

print(accuracy_score(y_test, pred_NB_cv))
print(classification_report(y_test, pred_NB_cv))

0.9246629659000793
                        precision    recall  f1-score   support

                 Books       0.96      0.88      0.92       586
Clothing & Accessories       0.94      0.94      0.94       466
           Electronics       0.91      0.91      0.91       506
             Household       0.91      0.95      0.93       964

              accuracy                           0.92      2522
             macro avg       0.93      0.92      0.92      2522
          weighted avg       0.93      0.92      0.92      2522



Berdasarkan nilai akurasi dari kedua model Machine Learning di atas dengan menggunakan Count Vectorizer, dapat dikatakan bahwa model Logictic Regression menghasilkan tingkat akurasi yang sedikit lebih tinggi daripada model Multinomial Naive Bayes.

### TF - IDF Vectorizer

TF-IDF (Term Frequency-Inverse Document Frequency) adalah metode vectorization yang digunakan dalam pemrosesan teks untuk mengukur pentingnya sebuah kata dalam sebuah dokumen dalam konteks koleksi dokumen yang lebih besar. Metode ini memberikan bobot yang lebih tinggi pada kata-kata yang muncul lebih sering dalam dokumen tertentu, tetapi jarang muncul di dokumen lain.

Dengan menggunakan TF-IDF, kita dapat mengonversi teks menjadi representasi numerik yang memperhitungkan pentingnya kata dalam dokumen-dokumen yang berbeda. Representasi ini dapat digunakan sebagai input untuk algoritma Machine Learning untuk memahami dan memprediksi pola dalam teks.

Kemudian Saya melakukan hal yang sama yaitu mencari TF - IDF setelah itu memakai hasil dari TF - IDF tersebut ke dalam 2 model Machine Learning yaitu Logistic Regression dengan Multinomial Naive Bayes

In [ ]:
tfidf_vector = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5)

tfidf_vector.fit(X_train)

X_train_tfidf_vector = tfidf_vector.transform(X_train)
X_test_tfidf_vector = tfidf_vector.transform(X_test)

X_train_tfidf_vector.shape, X_test_tfidf_vector.shape

((10084, 28146), (2522, 28146))

### Logistic Regression With TF - IDF Vector

In [ ]:
lr_tf = LogisticRegression(max_iter=5000)
lr_tf.fit(X_train_tfidf_vector, y_train)
pred_tf = lr_tf.predict(X_test_tfidf_vector)

print(accuracy_score(y_test, pred_tf))
print(classification_report(y_test, pred_tf))

0.9544012688342586
                        precision    recall  f1-score   support

                 Books       0.97      0.93      0.95       586
Clothing & Accessories       0.97      0.95      0.96       466
           Electronics       0.95      0.93      0.94       506
             Household       0.94      0.98      0.96       964

              accuracy                           0.95      2522
             macro avg       0.96      0.95      0.95      2522
          weighted avg       0.95      0.95      0.95      2522



### Multinomial Naive Bayes With TF - IDF Vector

In [ ]:
pipeline = Pipeline([('tfidf_vector', TfidfVectorizer(max_df=100)),
                    ('mnb', MultinomialNB())])

pipeline.fit(X_train, y_train)
pred_NB_tf = pipeline.predict(X_test)

print(accuracy_score(y_test, pred_NB_tf))
print(classification_report(y_test, pred_NB_tf))

0.9008723235527359
                        precision    recall  f1-score   support

                 Books       0.96      0.88      0.92       586
Clothing & Accessories       0.97      0.85      0.90       466
           Electronics       0.95      0.83      0.89       506
             Household       0.83      0.98      0.90       964

              accuracy                           0.90      2522
             macro avg       0.93      0.88      0.90      2522
          weighted avg       0.91      0.90      0.90      2522



Dapat dilihat bahwa nilai akurasi model Logistic Regression dengan menggunakan TF - IDF Vector menghasilkan tingkat akurasi yang lebih tinggi daripada Multinomial Naive Bayes.

### Conclusion
Mari bandingkan antara Logistic Regression menggunakan Count Vectorizer dengan Logistic Regression menggunakan TF - IDF

Accuracy LR dengan Count Vectorizer sebesar: 0.9504361617763679

Accuracy LR dengan TF - IDF Vectorizer sebesar: 0.9544012688342586

Akurasi LR dengan TF - IDF lebih tinggi daripada dengan Count Vectorizer, namun perbedaan akurasi yang dihasilkan dari kedua model ini bisa dikatakan tidak terlalu signifikan.

# BERT MODELLING

### Import Library

Disini Saya melakukan download package yang bernama 'transformers' serta melakukan import beberapa library lagi

In [ ]:
!pip install transformers

import torch
from torch import nn
from torch.optim import Adam

from transformers import BertTokenizer
from transformers import BertModel
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s eta 0:00:00


BERT menggunakan arsitektur Transformer yang terdiri dari lapisan-lapisan self-attention. Self-attention memungkinkan model untuk fokus pada bagian penting dari teks saat memprosesnya, memungkinkan pemahaman yang lebih baik tentang hubungan antara kata-kata dalam teks.

Model BERT memiliki beberapa varian dengan ukuran yang berbeda, seperti BERT-base, BERT-large, dan varian lainnya. BERT-base memiliki 12 lapisan self-attention dan total sekitar 110 juta parameter. Sementara itu, BERT-large memiliki 24 lapisan self-attention dan sekitar 340 juta parameter. Model yang lebih besar cenderung memiliki kemampuan yang lebih baik dalam memahami teks, tetapi juga memerlukan lebih banyak sumber daya komputasi untuk melatih dan menerapkannya.

Salah satu fitur utama dari BERT adalah kemampuannya untuk menghasilkan representasi kata-kata yang kontekstual, yang berarti representasi kata-kata itu bergantung pada konteks kalimat di mana kata-kata tersebut muncul. Representasi ini memperhitungkan kata-kata sebelum dan sesudahnya dalam kalimat untuk memahami arti kata tersebut dengan lebih baik.

Disini saya menggunakan ukuran model berupa 'bert-base-cased' kemudian mengubah label yang tadinya dalam bentuk object akan di ubah ke dalam numerik.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'Books':0,
          'Household':1,
          'Electronics':2,
          'Clothing & Accessories':3
          }

Kelas Dataset di bawah ini digunakan untuk mengorganisir data dalam format yang sesuai dengan kebutuhan pemrosesan dan pelatihan model menggunakan PyTorch. Dalam hal ini, dataset terdiri dari teks dan label kelas yang siap digunakan dalam pelatihan atau evaluasi model Machine Learning.

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['label']]
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['clean_text']]

    # Metode ini mengembalikan nilai dari atribut labels, yaitu daftar label kelas dalam bentuk representasi numerik.
    def classes(self):
        return self.labels

    # Metode ini mengembalikan panjang dataset, yaitu jumlah data yang ada dalam dataset.
    def __len__(self):
        return len(self.labels)

    # Metode ini mengembalikan batch dari label kelas dalam bentuk array numpy.
    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    # Metode ini mengembalikan batch dari teks dalam bentuk objek tensor PyTorch.
    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    # Metode ini mengembalikan batch teks dan label kelas berdasarkan indeks idx.
    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

Code di bawah ini berguna untuk membagi data ke train, validation dan test dengan pembagian data train sebanyak 80%, data validation sebanyak 10% dan data test sebanyak 10%

In [ ]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df1.sample(frac=1, random_state=42),
                                     [int(.8*len(df1)), int(.9*len(df1))])

print(len(df_train),len(df_val), len(df_test))

10084 1261 1261


Kode di bawah mendefinisikan kelas BertClassifier, yang merupakan turunan dari kelas nn.Module dalam framework PyTorch. Kelas ini bertujuan untuk mengimplementasikan model klasifikasi menggunakan arsitektur BERT.

In [ ]:
class BertClassifier(nn.Module):
    # Pada bagian ini, metode konstruktor melakukan inisialisasi beberapa komponen model, yaitu BERT, lapisan dropout, lapisan linear, dan fungsi aktivasi ReLU.
    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        # Bagian ini menginisialisasi model BERT menggunakan pre-trained model bert-base-cased.
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5) # input ukuran 768 dan output ukuran 5.
        self.relu = nn.ReLU()

    # Metode forward digunakan untuk mengimplementasikan aliran data maju (forward pass) melalui model.
    # Hasil akhirnya adalah output model yang merupakan representasi hasil klasifikasi.
    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

Kode di bawah adalah sebuah fungsi train yang digunakan untuk melatih (training) model klasifikasi menggunakan data pelatihan (train_data) dan data validasi (val_data). Fungsi ini melakukan pelatihan model dengan menggunakan algoritma backpropagation dan optimisasi stokastik gradien.

Di function ini juga Saya menggunakan 2 Hyperparameter yang bisa di tuning yaitu di batch_size dan juga di learning rate, jadi untuk meningkatkan hasil akurasi anda bisa saja mengubah kedua hyperparameter ini untuk mendapatkan hasil yang lebih memuaskan.

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

EPOCHS = 5
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 5042/5042 [17:08<00:00,  4.90it/s]


Epochs: 1 | Train Loss:  0.338                 | Train Accuracy:  0.792                 | Val Loss:  0.124                 | Val Accuracy:  0.947


100%|██████████| 5042/5042 [17:07<00:00,  4.91it/s]


Epochs: 2 | Train Loss:  0.119                 | Train Accuracy:  0.945                 | Val Loss:  0.109                 | Val Accuracy:  0.947


100%|██████████| 5042/5042 [17:08<00:00,  4.90it/s]


Epochs: 3 | Train Loss:  0.084                 | Train Accuracy:  0.962                 | Val Loss:  0.083                 | Val Accuracy:  0.960


100%|██████████| 5042/5042 [17:07<00:00,  4.91it/s]


Epochs: 4 | Train Loss:  0.060                 | Train Accuracy:  0.974                 | Val Loss:  0.087                 | Val Accuracy:  0.956


100%|██████████| 5042/5042 [17:08<00:00,  4.90it/s]


Epochs: 5 | Train Loss:  0.044                 | Train Accuracy:  0.981                 | Val Loss:  0.083                 | Val Accuracy:  0.960


Kode di bawah adalah sebuah fungsi evaluate yang digunakan untuk mengevaluasi performa model klasifikasi pada data uji (test_data). Fungsi ini menghitung akurasi prediksi model terhadap data uji.

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df_test)

Test Accuracy:  0.947


Akurasi yang dihasilkan oleh model BERT ini sebesar 0.947 menurut saya hasil ini sudah cukup memuaskan karna tingakt akurasi yang sudah sangat tinggi. Jika ingin mencapai akurasi yang lebih tinggi lagi, bisa dilakukan dengan cara mengubah nilai dari hyperparameter jadi bisa dilakukan tuning sesuka hati.